In [1]:
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px


In [3]:
# -----------------------------
# Load Data
# -----------------------------

file_path = r"E:\ViSTA Project Crop extent Mapping\Maize Fingerprint_2015_EC.csv"

df = pd.read_csv(file_path)

# valid GPS points
df = df[(df["MLATITUDE_N"] != 0) & (df["MLONGITUDE_E"] != 0)]

In [4]:
# -----------------------------
# Regional statistics
# -----------------------------

region_stats = df.groupby("MREGION_NAME").agg(
    total_maize_area=("TOTAL_MAIZE_AREA","sum"),
    improved_maize=("AREA_UNDER_IMPROVED_MAIZE","sum"),
    samples=("MHHID","count")
).reset_index()

# -----------------------------
# Plotly Charts
# -----------------------------

bar_chart = px.bar(
    region_stats,
    x="MREGION_NAME",
    y=["total_maize_area","improved_maize"],
    title="Maize Production by Region",
    barmode="group"
)

scatter_map = px.scatter_mapbox(
    df,
    lat="MLATITUDE_N",
    lon="MLONGITUDE_E",
    hover_name="MREGION_NAME",
    zoom=5,
    height=600
)

scatter_map.update_layout(mapbox_style="open-street-map")

C:\Users\PRE\AppData\Local\Temp\ipykernel_17132\3774054039.py:23: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [6]:
# -----------------------------
# Dash App Layout
# -----------------------------

app = dash.Dash(__name__)

app.layout = html.Div([

    html.H1("Maize Fingerprint Dashboard (Ethiopia)", 
            style={'textAlign':'center'}),

    dcc.Graph(
        figure=bar_chart
    ),

    dcc.Graph(
        figure=scatter_map
    )

])

In [8]:
from dash import dcc, html, Input, Output

app.layout = html.Div([

    html.H1("Maize Crop Monitoring Dashboard"),

    dcc.Dropdown(
        id='region_filter',
        options=[{'label': r, 'value': r} for r in df["MREGION_NAME"].unique()],
        value=df["MREGION_NAME"].unique()[0]
    ),

    dcc.Graph(id='maize_chart'),

])

In [9]:
@app.callback(
    Output("maize_chart", "figure"),
    Input("region_filter", "value")
)
def update_chart(selected_region):

    filtered = df[df["MREGION_NAME"] == selected_region]

    stats = filtered.groupby("MZONE_NAME").agg(
        maize_area=("TOTAL_MAIZE_AREA","sum")
    ).reset_index()

    fig = px.bar(stats,
                 x="MZONE_NAME",
                 y="maize_area",
                 title=f"Maize Area in {selected_region}")

    return fig

In [10]:
import plotly.express as px

heatmap = px.density_mapbox(
    df,
    lat="MLATITUDE_N",
    lon="MLONGITUDE_E",
    radius=20,
    zoom=5,
    mapbox_style="open-street-map"
)

C:\Users\PRE\AppData\Local\Temp\ipykernel_17132\4200932555.py:3: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [13]:
app.layout = html.Div([
    html.H1("Ethiopia Maize Dashboard", style={'textAlign': 'center'}),

    html.H2("Maize Production by Region"),
    dcc.Graph(
        figure=bar_chart
    ),

    html.H2("Maize Sample Locations"),
    dcc.Graph(
        figure=scatter_map
    )
])

In [ ]:
# -----------------------------
# Run Server
# -----------------------------

if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
